Members :


1.   PRAKASH PRASAD - 2022ac05256@wilp.bits-pilani.ac.in
2.   Mulla Akib Javed - 2022ac05631@wilp.bits-pilani.ac.in
3.   AMIT KUMAR SRIVASTAVA - 2022ac05292@wilp.bits-pilani.ac.in
4.   MRUNAL MADHUKAR AHIRE - 2022ac05228@wilp.bits-pilani.ac.in




Dataset link : https://huggingface.co/datasets/nvidia/HelpSteer



In [2]:
!pip install git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git -q
!pip install bitsandbytes datasets accelerate loralib -q
# !pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/cu11.8/torch_stable.html

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
from huggingface_hub import notebook_login
notebook_login()
token = '' # add token from hf

In [4]:
import warnings
import pandas as pd
warnings.filterwarnings("ignore")
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
import torch
from datasets import load_dataset
import time
import torch
import torch.nn as nn
import torch.optim as optim

# from torch.nn.parallel import DistributedDataParallel as DDP
# from accelerate import Accelerator
# from accelerate.utils import write_basic_config

from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, RobertaTokenizer, GPT2Tokenizer, \
    GPT2LMHeadModel, AutoTokenizer, AutoModelForCausalLM, AdamW
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, decoders, processors

# torch.random.manual_seed(0)
# torch.cuda.set_device(0)
# # torch.cuda.set_device(1)
# # torch.cuda.set_device(2)
# # torch.cuda.set_device(3)
# torch.cuda.is_available()


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling
from datasets import load_dataset
import pandas as pd

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
# load dataset
ds = load_dataset("nvidia/HelpSteer2")
train = ds['train'] # len(train) = 20324 (95%)
val = ds['validation']     # len(val) = 1038 (5%)

train_pd = pd.DataFrame(train)
val_pd = pd.DataFrame(val)
train_pd.head()
val_pd.head()

train_df = train_pd.copy()
val_df = val_pd.copy()

## 1. no preprocessing needed for llm , lm preprocessing for lstm: [1 Mark]

In [ ]:
train_pd = train_pd[:500]
val_pd = val_pd[:50]

In [ ]:
# Tokenize the prompts and answers
tokenizer = get_tokenizer("basic_english")

def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

# Build vocabulary
vocab = build_vocab_from_iterator(yield_tokens(train_pd['prompt'].tolist() + train_pd['response'].tolist()),
                                  specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

# Convert texts to sequences
def text_pipeline(text):
    return vocab(tokenizer(text))

train_prompt_sequences = [text_pipeline(text) for text in train_pd['prompt']]
train_answer_sequences = [text_pipeline(text) for text in train_pd['response']]
val_prompt_sequences = [text_pipeline(text) for text in val_pd['prompt']]
val_answer_sequences = [text_pipeline(text) for text in val_pd['response']]

max_len_prompt = max(len(seq) for seq in train_prompt_sequences)
max_len_answer = max(len(seq) for seq in train_answer_sequences)
print('max token length of prompts and response: ', max_len_prompt, max_len_answer)
# Vocabulary size
vocab_size = len(vocab)
print('Vocabulary size from train prompt + response ', vocab_size)

# Pad the sequences
def pad_sequences(sequences, max_len):
    return pad_sequence([torch.tensor(seq[:max_len]) for seq in sequences], batch_first=True,
                        padding_value=vocab["<unk>"])

train_prompt_sequences = pad_sequences(train_prompt_sequences, max_len_prompt)
train_answer_sequences = pad_sequences(train_answer_sequences, max_len_answer)
val_prompt_sequences = pad_sequences(val_prompt_sequences, max_len_prompt)
val_answer_sequences = pad_sequences(val_answer_sequences, max_len_answer)

max token length of prompts and response:  548 1045
Vocabulary size from train prompt + response  7095


In [ ]:
train_prompt_sequences.shape, train_answer_sequences.shape, val_prompt_sequences.shape, val_answer_sequences.shape

(torch.Size([300, 548]),
 torch.Size([300, 1045]),
 torch.Size([50, 387]),
 torch.Size([50, 808]))

## 2.Explain and implement how LLM (2B to 8B parameters) over simple LM can help improve the system accuracy. [1 Mark]

In [ ]:
# Seq 2 Seq LM using lstm on Pytorch
import torch.nn as nn

class Seq2Seq(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_units):
        super(Seq2Seq, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.encoder_lstm = nn.LSTM(embedding_dim, hidden_units, batch_first=True)
        self.decoder_lstm = nn.LSTM(embedding_dim, hidden_units, batch_first=True)
        self.fc = nn.Linear(hidden_units, vocab_size)

    def forward(self, encoder_input, decoder_input, hidden):
        encoder_embedded = self.embedding(encoder_input)
        encoder_output, (hidden, cell) = self.encoder_lstm(encoder_embedded, hidden)

        decoder_embedded = self.embedding(decoder_input)
        decoder_output, (hidden, cell) = self.decoder_lstm(decoder_embedded, (hidden, cell))

        output = self.fc(decoder_output)
        return output, (hidden, cell)

embedding_dim = 128
hidden_units = 256

model = Seq2Seq(vocab_size, embedding_dim, hidden_units)

In [ ]:

# Hyperparameters
num_epochs = 20
learning_rate = 0.001
batch_size = 32

# Define the loss and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=vocab["<unk>"])
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

device='cpu' # using cpu since gpu gives memory error
# Move model to GPU
# device_ids = [0, 1, 2, 3]  # Replace with desired GPU IDs
# model = nn.DataParallel(model, device_ids=device_ids)
# model.to(f'cuda:{device_ids[0]}')  # Move the model to the first GPU

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# # input_tensor = input_tensor.to(device)
# model = model.to(device)

# scaler = torch.cuda.amp.GradScaler()
# Prepare the target data for the decoder (shifted one time step) !!!
def prepare_target_sequences(sequences):
    inputs = sequences[:, :-1]
    targets = sequences[:, 1:]
    return inputs, targets

train_answer_sequences_input, train_answer_sequences_output = prepare_target_sequences(train_answer_sequences)
val_answer_sequences_input, val_answer_sequences_output = prepare_target_sequences(val_answer_sequences)

# Convert to PyTorch tensors and move to device
train_prompt_sequences = train_prompt_sequences.to(device)
train_answer_sequences_input = train_answer_sequences_input.to(device)
train_answer_sequences_output = train_answer_sequences_output.to(device)
val_prompt_sequences = val_prompt_sequences.to(device)
val_answer_sequences_input = val_answer_sequences_input.to(device)
val_answer_sequences_output = val_answer_sequences_output.to(device)


In [ ]:
# Training loop
init = time.time()
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    output, _ = model(train_prompt_sequences, train_answer_sequences_input, None)
#     loss = criterion(output.view(-1, vocab_size), train_answer_sequences_output.view(-1)) # ask
    loss = criterion(output.reshape(-1, vocab_size), train_answer_sequences_output.reshape(-1))

    loss.backward()
    optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")

time.time() - init

Epoch [1/50], Loss: 8.87558364868164
Epoch [2/50], Loss: 8.849912643432617
Epoch [3/50], Loss: 8.822985649108887
Epoch [4/50], Loss: 8.792208671569824
Epoch [5/50], Loss: 8.754260063171387
Epoch [6/50], Loss: 8.703964233398438
Epoch [7/50], Loss: 8.631867408752441
Epoch [8/50], Loss: 8.517773628234863
Epoch [9/50], Loss: 8.311432838439941
Epoch [10/50], Loss: 7.934554100036621
Epoch [11/50], Loss: 7.5272321701049805
Epoch [12/50], Loss: 7.2270894050598145
Epoch [13/50], Loss: 7.004649639129639
Epoch [14/50], Loss: 6.82792854309082
Epoch [15/50], Loss: 6.6866374015808105
Epoch [16/50], Loss: 6.579782009124756
Epoch [17/50], Loss: 6.508011341094971
Epoch [18/50], Loss: 6.469689846038818
Epoch [19/50], Loss: 6.459895610809326
Epoch [20/50], Loss: 6.468969345092773
Epoch [21/50], Loss: 6.484289169311523
Epoch [22/50], Loss: 6.495165824890137
Epoch [23/50], Loss: 6.496269226074219
Epoch [24/50], Loss: 6.487391948699951
Epoch [25/50], Loss: 6.471380233764648
Epoch [26/50], Loss: 6.4520812034

466.6952950954437

In [ ]:
torch.save(model, 'lstm_lm_model.pth')

In [ ]:
model = torch.load('lstm_lm_model.pth')
model.eval()

Seq2Seq(
  (embedding): Embedding(7095, 128)
  (encoder_lstm): LSTM(128, 256, batch_first=True)
  (decoder_lstm): LSTM(128, 256, batch_first=True)
  (fc): Linear(in_features=256, out_features=7095, bias=True)
)

In [ ]:
# Validation
model.eval()
with torch.no_grad():
    val_output, _ = model(val_prompt_sequences, val_answer_sequences_input, None)
#     val_loss = criterion(val_output.view(-1, vocab_size), val_answer_sequences_output.view(-1))
    val_loss = criterion(val_output.reshape(-1, vocab_size), val_answer_sequences_output.reshape(-1))
    print(f"Validation Loss: {val_loss.item()}")

Validation Loss: 6.111302852630615


In [ ]:
def generate_simple_lm_response(prompt, model, vocab, max_len_answer):
    model.eval()
    with torch.no_grad():
        prompt_sequence = pad_sequence([torch.tensor(text_pipeline(prompt))[:max_len_prompt]], batch_first=True, padding_value=vocab["<unk>"]).to(device)
        decoder_input = torch.zeros((1, 1), dtype=torch.long).to(device)

        hidden = None
        decoded_sentence = []

        for _ in range(max_len_answer):
            output, hidden = model(prompt_sequence, decoder_input, hidden)
            sampled_token_index = output.argmax(2)[:, -1].item()
            sampled_word = vocab.lookup_token(sampled_token_index)
            decoded_sentence.append(sampled_word)
            if sampled_word == '<end>':
                break
            decoder_input = torch.cat([decoder_input, torch.tensor([[sampled_token_index]], dtype=torch.long).to(device)], dim=1)

        return ' '.join(decoded_sentence)

In [ ]:
# Generate an answer - simple LM
prompt_ = val_pd['prompt'][9]
print(prompt_)
print(generate_simple_lm_response(prompt_, model, vocab, max_len_answer))

Please give us creative product ideas related to air fresheners.
, and , and , and , and , and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and the and t

In [ ]:
llm_model_name = "meta-llama/Meta-Llama-3-8B"
llm_tokenizer = AutoTokenizer.from_pretrained(llm_model_name, add_bos_token=False, add_eos_token=True)
llm_tokenizer.pad_token = llm_tokenizer.eos_token
llm_model = AutoModelForCausalLM.from_pretrained(llm_model_name)
device = 'cpu'
print(device)
llm_model.to(device)

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

cpu


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

In [ ]:
def generate_llm_responses(text_):
    inputs = llm_tokenizer.encode_plus(text_, return_tensors='pt', padding=True, truncation=True).to(device)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    outputs = llm_model.generate(input_ids, attention_mask=attention_mask, max_length=140,
                                 num_return_sequences=1, pad_token_id=llm_tokenizer.eos_token_id)
    response = llm_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

prompt_ = val_pd['prompt'][9]
print('prompt: ', prompt_)
print('meta lamma tokeniser output: ', llm_tokenizer.tokenize(prompt_))
print('generated response: ', generate_llm_responses(prompt_))

prompt:  Please give us creative product ideas related to air fresheners.
meta lamma tokeniser output:  ['Please', 'Ġgive', 'Ġus', 'Ġcreative', 'Ġproduct', 'Ġideas', 'Ġrelated', 'Ġto', 'Ġair', 'Ġfresh', 'eners', '.']
generated response:  Please give us creative product ideas related to air fresheners. I am looking for unique, new and innovative ideas for air fresheners. These ideas should be innovative and creative.


## 3.	Explain and implement how specific tokenizer and model can work well with the mentioned dataset. [1 Mark]

In [ ]:
bpe_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
bpe_tokenized_prompt_ = bpe_tokenizer.tokenize(prompt_)
print("BPE Tokenized Prompts: ", bpe_tokenized_prompt_)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

BPE Tokenized Prompts:  ['Please', 'Ġgive', 'Ġus', 'Ġcreative', 'Ġproduct', 'Ġideas', 'Ġrelated', 'Ġto', 'Ġair', 'Ġfres', 'hen', 'ers', '.']


In [ ]:
char_tokenizer = Tokenizer(models.WordLevel(unk_token="[UNK]"))
char_tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
char_tokenizer.decoder = decoders.WordPiece(prefix="##")

vocab_size_=vocab_size
# Train the tokenizer on the dataset
trainer = trainers.WordLevelTrainer(vocab_size=vocab_size_, special_tokens=["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"])
char_tokenizer.train_from_iterator(train_df['prompt'], trainer)

# Tokenize using the character-level tokenizer
char_tokenized_prompt_ = char_tokenizer.encode(prompt).tokens
print("Character-Level Tokenized Prompts: ", char_tokenized_prompt_)

Character-Level Tokenized Prompts:  ['explain', 'master', 'slave', 'replication', '[UNK]']


In [ ]:
# Generate predictions using BPE tokenizer
inputs = bpe_tokenizer.encode_plus(prompt_, return_tensors='pt', padding=True, truncation=True)
input_ids = inputs['input_ids'].to(device)
attention_mask = inputs['attention_mask'].to(device)
outputs = llm_model.generate(input_ids, attention_mask=attention_mask, max_length=140,
                             num_return_sequences=1, pad_token_id=bpe_tokenizer.eos_token_id)
response = bpe_tokenizer.decode(outputs[0], skip_special_tokens=True)
print('prompt: ', prompt_)
# print('\n')
print('bpe encoded response: ', response)

prompt:  Please give us creative product ideas related to air fresheners.
bpe encoded response:  Please give us creative product ideas related to air fresheners. next's for is for� for� us. next's for is for� for's us. next's for is for� for with us. next's for is for� for The us. next's for is for� for was us. next's for is for� for " us. next's for is for� for at us. next's for is for� for it us. next's for is for� forw us. next's for is for� for technology us. next's for is for� forE us. next's for is


In [ ]:
# # Generate predictions using BPE tokenizer
# inputs = bpe_tokenizer.encode_plus(prompt_, return_tensors='pt', padding=True, truncation=True)
# input_ids = inputs['input_ids']
# attention_mask = inputs['attention_mask']
# outputs = llm_model.generate(input_ids, attention_mask=attention_mask, max_length=140,
#                              num_return_sequences=1, pad_token_id=llm_tokenizer.eos_token_id)
# response = llm_tokenizer.decode(outputs[0], skip_special_tokens=True)
# print('prompt: ', prompt_)
# print('\n')
# print('bpe encoded response: ', response)

In [ ]:
# Generate predictions using char tokenizer
encoding = char_tokenizer.encode(prompt_)
input_ids = torch.tensor([encoding.ids], dtype=torch.long).to(device)
attention_mask = torch.ones_like(input_ids).to(device)  # Dummy attention mask, since char_tokenizer doesn't provide it
outputs = llm_model.generate(input_ids, attention_mask=attention_mask, max_length=140,
                             num_return_sequences=1, pad_token_id=llm_tokenizer.eos_token_id)
response = char_tokenizer.decode(outputs[0].cpu().numpy().tolist())
print('prompt: ', prompt_)
print('\n')
print('char level encoded response: ', response)

prompt:  Please give us creative product ideas related to air fresheners.


char level encoded response:  Please give us creative product ideas related to air. to. like. to. like..........................................................


## 4.	Explain and implement how sentiment analysis can be used to analyze user questions or contexts. [1 Mark]

In [ ]:
from transformers import pipeline

# Initialize the sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

# A list of example questions is created to demonstrate how sentiment analysis works.
user_texts = [
    "I'm really happy with the new product!",
    "I had a terrible experience with the customer service.",
    "Can you help me with my issue?",
    "The latest update is okay, but it could be better."
]

# Each question is analyzed for sentiment, and the results are printed out.
for text in user_texts:
    sentiment = sentiment_pipeline(text) # It takes a text input, runs it through the sentiment analysis pipeline, and returns the result.
    print(f"Text: {text}")
    print(f"Sentiment: {sentiment}")
    print()

## 5.	Implement fine-tuning for an open-source model to improve the results. [3 Marks]

In [ ]:
def tokenize_function(examples):
    inputs = llm_tokenizer(examples['prompt'], add_special_tokens=True, max_length=512, padding="max_length", truncation=True, return_tensors="pt")
    outputs = llm_tokenizer(examples['response'], add_special_tokens=True, max_length=512, padding="max_length", truncation=True, return_tensors="pt")
    inputs['labels'] = outputs['input_ids']
    return inputs

dataset = load_dataset("nvidia/HELPSTEER2")
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=dataset["train"].column_names)
tokenized_datasets.set_format("torch")
train_dataset = tokenized_datasets["train"].select(range(20))
eval_dataset = tokenized_datasets["validation"].select(range(20))


Map:   0%|          | 0/20324 [00:00<?, ? examples/s]

Map:   0%|          | 0/1038 [00:00<?, ? examples/s]

In [ ]:
# Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=llm_tokenizer, mlm=False)

In [ ]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 20
})

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    use_cpu=True,  # Ensures training is on CPU
)

In [ ]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 20
})

In [ ]:
init = time.time()
# Trainer
trainer = Trainer(
    model=llm_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Train model
trainer.train()
time.time()-init

Epoch,Training Loss,Validation Loss
1,No log,5.521920
2,No log,4.785708
3,No log,4.745604


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


889.6026239395142

In [ ]:
init = time.time()
model_save_path = "sft_finetuned_llm_model.pth"
torch.save(llm_model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")
time.time() - init

Model saved to sft_finetuned_llm_model.pth


## 6.	Show 5-10 working examples that show improvements of the accuracy of the fine-tuned model. [3 Marks]

In [ ]:
# loading original model and finding responses on old and sft trained model on 6 prompts in val df

In [ ]:
m1_tokeniser = AutoTokenizer.from_pretrained(llm_model_name, add_bos_token=False, add_eos_token=True)
m1_tokeniser.pad_token = m1_tokeniser.eos_token
m1_model = AutoModelForCausalLM.from_pretrained(llm_model_name)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# prompt 1

In [ ]:
def generate_llm_responses(text_):
    inputs = llm_tokenizer.encode_plus(text_, return_tensors='pt', padding=True, truncation=True).to(device)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    outputs = llm_model.generate(input_ids, attention_mask=attention_mask, max_length=140,
                                 num_return_sequences=1, pad_token_id=llm_tokenizer.eos_token_id)
    response = llm_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

prompt_ = 'Please give us creative product ideas related to air fresheners.'
print('prompt: ', prompt_)
print('meta lamma tokeniser output: ', llm_tokenizer.tokenize(prompt_))
print('generated response: ', generate_llm_responses(prompt_))

prompt:  Please give us creative product ideas related to air fresheners.
meta lamma tokeniser output:  ['Please', 'Ġgive', 'Ġus', 'Ġcreative', 'Ġproduct', 'Ġideas', 'Ġrelated', 'Ġto', 'Ġair', 'Ġfresh', 'eners', '.']
generated response:  Please give us creative product ideas related to air fresheners.


In [ ]:
def generate_m1_responses(text_):
    inputs =m1_tokeniser.encode_plus(text_, return_tensors='pt', padding=True, truncation=True).to(device)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    outputs = m1_model.generate(input_ids, attention_mask=attention_mask, max_length=140,
                                 num_return_sequences=1, pad_token_id=m1_tokeniser.eos_token_id)
    response = m1_tokeniser.decode(outputs[0], skip_special_tokens=True)
    return response

prompt_ = 'Please give us creative product ideas related to air fresheners.'
print('prompt: ', prompt_)
print('meta lamma tokeniser output: ', m1_tokeniser.tokenize(prompt_))
print('generated response: ', generate_m1_responses(prompt_))

prompt:  Please give us creative product ideas related to air fresheners.
meta lamma tokeniser output:  ['Please', 'Ġgive', 'Ġus', 'Ġcreative', 'Ġproduct', 'Ġideas', 'Ġrelated', 'Ġto', 'Ġair', 'Ġfresh', 'eners', '.']
generated response:  Please give us creative product ideas related to air fresheners. We are looking for creative ideas that can be turned into a product.
I'm looking for an air freshener with a unique scent. I need it to be something that I can sell to people who have dogs and need to freshen their homes.
I am looking for a creative product idea related to air fresheners. I need it to be something that I can sell to people who are looking for a unique way to freshen their homes.
I'm looking for a creative product idea related to air fresheners. I need it to be something that I can sell to people who are looking for a unique way to freshen their homes


In [ ]:
# prompt 2

In [ ]:
prompt_ = val_df['prompt'][14]
print('prompt: ', prompt_)
# print('meta lamma tokeniser output: ', llm_tokenizer.tokenize(prompt_))
print('generated response: ', generate_llm_responses(prompt_))

prompt:  what is SoP for studying abroad students

generated response:  what is SoP for studying abroad students



In [ ]:
prompt_ = val_df['prompt'][14]
print('prompt: ', prompt_)
# print('meta lamma tokeniser output: ', m1_tokeniser.tokenize(prompt_))
print('generated response: ', generate_m1_responses(prompt_))

prompt:  what is SoP for studying abroad students

generated response:  what is SoP for studying abroad students
The SoP for studying abroad students is a statement of purpose that helps you to get admission to a university. It is a document that you write to explain why you want to study abroad and what you hope to achieve by doing so. It should be written in a clear and concise manner, and it should be tailored to the specific university you are applying to. The SoP for studying abroad students is an important part of the application process, and it can help you to stand out from other applicants.
What is a SoP for studying abroad students?
A SoP for studying abroad students is a statement of purpose that helps students to explain their motivation for


In [ ]:
# prompt 3

In [ ]:
prompt_ = val_df['prompt'][18]
print('prompt: ', prompt_)
# print('meta lamma tokeniser output: ', llm_tokenizer.tokenize(prompt_))
print('generated response: ', generate_llm_responses(prompt_))

prompt:  Give me caption ideas that are somber and poetic. The caption is for a sinister looking woman with horns
generated response:  Give me caption ideas that are somber and poetic. The caption is for a sinister looking woman with horns


In [ ]:
prompt_ = val_df['prompt'][18]
print('prompt: ', prompt_)
# print('meta lamma tokeniser output: ', m1_tokeniser.tokenize(prompt_))
print('generated response: ', generate_m1_responses(prompt_))

prompt:  Give me caption ideas that are somber and poetic. The caption is for a sinister looking woman with horns
generated response:  Give me caption ideas that are somber and poetic. The caption is for a sinister looking woman with horns. She is standing in front of a dark, ominous, tree.
I think you need to give more context. Is she a demon? A witch? A vampire? A ghost? A zombie? A goth? A succubus? A demoness? A succubus? A succubus? A succubus? A succubus? A succubus? A succubus? A succubus? A succubus? A succubus? A succubus? A succubus? A succubus? A succubus?


In [ ]:
# prompt 4

In [ ]:
prompt_ = val_df['prompt'][24]
print('prompt: ', prompt_)
# print('meta lamma tokeniser output: ', llm_tokenizer.tokenize(prompt_))
print('generated response: ', generate_llm_responses(prompt_))

prompt:  Please provide the equation to calculate the moment of inertia of a trapezium
generated response:  Please provide the equation to calculate the moment of inertia of a trapezium


In [ ]:
prompt_ = val_df['prompt'][24]
print('prompt: ', prompt_)
# print('meta lamma tokeniser output: ', llm_tokenizer.tokenize(prompt_))
print('generated response: ', generate_m1_responses(prompt_))

prompt:  Please provide the equation to calculate the moment of inertia of a trapezium
generated response:  Please provide the equation to calculate the moment of inertia of a trapezium about the centroidal axis.
The moment of inertia of a trapezium about the centroidal axis can be calculated using the formula:
I = \frac{bh^3}{12} + \frac{dh^3}{12}
I = moment of inertia of the trapezium
b = base length
h = height of the trapezium
d = distance from the centroidal axis to the base of the trapezium


In [ ]:
# prompt 5

In [ ]:
prompt_ = val_df['prompt'][60]
print('prompt: ', prompt_)
# print('meta lamma tokeniser output: ', llm_tokenizer.tokenize(prompt_))
print('generated response: ', generate_llm_responses(prompt_))

prompt:  What are the different components of a business plan
generated response:  What are the different components of a business plan


In [ ]:
prompt_ = val_df['prompt'][60]
print('prompt: ', prompt_)
# print('meta lamma tokeniser output: ', llm_tokenizer.tokenize(prompt_))
print('generated response: ', generate_m1_responses(prompt_))

prompt:  What are the different components of a business plan
generated response:  What are the different components of a business plan?
What are the different components of a business plan?
The key components of a business plan include:
What are the 4 elements of a business plan?
The four key elements of a business plan are:
The Executive Summary. The executive summary is the first section of the business plan.
The Company Description.
The Market Analysis.
The Marketing Plan.
The Company Organization.
The Service or Product Line.
The Marketing Strategy.
The Competition Analysis.
What is the most important part of a business plan?
The executive summary is the most important part of your business plan. It is the first thing that potential investors will see, and it will determine whether they will read


In [ ]:
# prompt 6

In [ ]:
prompt_ = val_df['prompt'][110]
print('prompt: ', prompt_)
# print('meta lamma tokeniser output: ', llm_tokenizer.tokenize(prompt_))
print('generated response: ', generate_llm_responses(prompt_))

prompt:  Explain Business plan legal use vs operating agreement
generated response:  Explain Business plan legal use vs operating agreement


In [ ]:
prompt_ = val_df['prompt'][110]
print('prompt: ', prompt_)
# print('meta lamma tokeniser output: ', llm_tokenizer.tokenize(prompt_))
print('generated response: ', generate_m1_responses(prompt_))

prompt:  Explain Business plan legal use vs operating agreement
generated response:  Explain Business plan legal use vs operating agreement
Explain Business plan legal use vs operating agreement
Explain Business plan legal use vs operating agreement
Explain Business plan legal use vs operating agreement
Explain Business plan legal use vs operating agreement
Explain Business plan legal use vs operating agreement
Explain Business plan legal use vs operating agreement
Explain Business plan legal use vs operating agreement
Explain Business plan legal use vs operating agreement
Explain Business plan legal use vs operating agreement
Explain Business plan legal use vs operating agreement
Explain Business plan legal use vs operating agreement
Explain Business plan legal use vs operating agreement
Explain Business plan legal use vs operating agreement


## 7.	Apply and fine-tune the generator model using Parameter-Efficient Fine-Tuning (PEFT). [3 Marks]

In [6]:
#imports
import warnings
warnings.filterwarnings("ignore")
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
torch.random.manual_seed(0)

#load the base model.
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
     token=token
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct", token=token)


# freezr the parameters.
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)


def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

# Create lora config.
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8, #attention dimension/rank
    lora_alpha=32, #alpha scaling
    target_modules=["qkv_proj"], #add the layers you to LoRA finetune
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

# 3072x9216
# 3072x8 8x9216

model = get_peft_model(model, config)
print_trainable_parameters(model)





config.json:   0%|          | 0.00/3.48k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

trainable params: 3145728 || all params: 3824225280 || trainable%: 0.0822579155169436


In [7]:
#iport the data and do the necessary processing to prepare the dataset for training.
import pandas as pd
splits = {'train': 'train.jsonl.gz', 'validation': 'validation.jsonl.gz'}
df = pd.read_json("hf://datasets/nvidia/HelpSteer/" + splits["train"], lines=True)
print(df.head())

from datasets import Dataset

# Convert the DataFrame to a Dataset
hf_dataset = Dataset.from_pandas(df)
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

def preprocess_function(examples):
    # Encode the prompts and responses. We add special tokens to both input and target.
    model_inputs = tokenizer(examples['prompt'], max_length=512, truncation=True, padding="max_length")

    # We need to specify that we're using the tokenizer for the targets to ensure it does not add special tokens
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['response'], max_length=128, truncation=True, padding="max_length")

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Apply the function to all splits in the dataset
tokenized_datasets = hf_dataset.map(preprocess_function, batched=True)
# Split the dataset into training and testing (or validation) sets
train_test_split = tokenized_datasets.train_test_split(test_size=0.1)
dataset_dict = {"train": train_test_split["train"], "test": train_test_split["test"]}
dataset_dict

                                              prompt  \
0  What are the three most important things to co...   
1  What are the three most important things to co...   
2  What are the three most important things to co...   
3  What are the three most important things to co...   
4  Background:\n<start of reference>\nFamily doct...   

                                            response  helpfulness  \
0  To build an assistive device to help an elderl...            3   
1  There are many different types of assistive de...            4   
2  When deciding what technology to use to build ...            4   
3  You can create an assistant device to help an ...            3   
4  Hi there! I'm Dr. Family, and I'm here to tell...            3   

   correctness  coherence  complexity  verbosity  
0            4          4           2          2  
1            3          3           2          3  
2            4          4           2          2  
3            3          3           2       

Map:   0%|          | 0/35331 [00:00<?, ? examples/s]

{'train': Dataset({
     features: ['prompt', 'response', 'helpfulness', 'correctness', 'coherence', 'complexity', 'verbosity', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 31797
 }),
 'test': Dataset({
     features: ['prompt', 'response', 'helpfulness', 'correctness', 'coherence', 'complexity', 'verbosity', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 3534
 })}

In [8]:
# start training.
import transformers

trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset_dict['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        warmup_steps=100,
        max_steps=200,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

# Create dir to save the model and save the model and tokenizer in that directory.
os.mkdir("/content/model_repo")
model.save_pretrained('/content/model_repo')
tokenizer.save_pretrained('/content/model_repo')


max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
1,3.417600
2,1.715700
3,2.173200
4,2.681000
5,2.328500
6,1.849100
7,2.691700
8,2.029000
9,1.865900
10,1.699100


('/content/model_repo/tokenizer_config.json',
 '/content/model_repo/special_tokens_map.json',
 '/content/model_repo/tokenizer.model',
 '/content/model_repo/added_tokens.json',
 '/content/model_repo/tokenizer.json')

In [9]:
# delete the model from runtime.
del model
del tokenizer
del trainer
torch.cuda.empty_cache()

In [10]:
#Load the model from local
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Set the local path where your model and tokenizer are stored
local_model_path = '/content/model_repo/'

# Create quantization config
quantization_config = BitsAndBytesConfig(llm_int8_enable_fp32_cpu_offload=True)

# Load the PEFT configuration from the local path
config = PeftConfig.from_pretrained(local_model_path)

# Load the base model and tokenizer with quantization config
model = AutoModelForCausalLM.from_pretrained(local_model_path, return_dict=True, quantization_config=quantization_config, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(local_model_path)

# Load the LoRA model adjustments from the same local path (assuming PEFT adjustments are saved in the same directory)
# Note: This assumes that the PeftModel class can handle loading from a local path; this may require additional implementation depending on the PEFT library specifics
model = PeftModel.from_pretrained(model, local_model_path)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
# check the output for an example.
batch = tokenizer("<|prompt|>\nWhat are the three most important things to consider when deciding what technology to use to build an assist device to help an elderly person with basic needs?<|end|>\n<|response|>", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50)

print('\n\n response \n ', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

You are not running the flash-attention implementation, expect numerical differences.




 response 
  <|prompt|>
What are the three most important things to consider when deciding what technology to use to build an assist device to help an elderly person with basic needs? <|response|>
When deciding on technology to build an assist device for an elderly person with basic needs, there are three critical factors to consider:

1. User-friendlinemen: The technology should be easy to use and understand for the elder


## 8.	Implement a function to evaluate the model's performance using metrics such as accuracy and F1 score. [1 Mark]

## 9.	Analyze the results to discuss the impact of PEFT on model performance and efficiency. [1 Mark]